# Proyecto Final: Clasificación de tópicos de interés

## Exploración de datos


CC5113 - Aprendizaje Automático Bayesiano

Profesor: Pablo Guerrero

Autor: Martín Cornejo

## Paquetes necesarios

In [123]:
import numpy as np
import matplotlib.pyplot as plt
import pdb
import itertools
import operator

import pandas as pd

from nltk.stem.snowball import SpanishStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [124]:
datos=pd.read_csv('data_format.csv')
print(datos.shape)
datos.head()

(138, 2)


,Texto,Interes
0,"Ojalá obliguen a Piñera a cerrar Punta Peuco, ...",False
1,Piñera para crear base de apoyo moderada a su ...,True
2,@CNNChile MEMORIA 2014 Adimark: Piñera termina...,True
3,PPK y Piñera en privado habrían conversado alg...,False
4,Bachelet entregará el gobierno de Chile a Piñera,True


## Limpiando strings

### Reemplazar tildes, caracteres especiales, todo a minúsculas

In [154]:
strings = datos.ix[:,0]

def formatear(strings):
    tildes = ['á','é','í','ó','ú', 'ñ']
    vocales = ['a','e','i','o','u', 'n']

    # tildes
    for idx, vocal in enumerate(vocales):
        strings = strings.str.replace(tildes[idx],vocal)

    # caracteres especiales menos la ñ
    strings = strings.str.replace('[^a-zñA-Z ]', "")

    # todo a minusculas
    strings = pd.Series(list(map(lambda x: x.lower(), strings)))
    
    return strings

def oracionToStrArr(strings):
    strings_arr = list(map(lambda x: x.split(), strings))
    #pdb.set_trace()
    strings_arr = list(itertools.chain.from_iterable(strings_arr))    
    return strings_arr

print(formatear(strings).head())
formated_array_data = oracionToStrArr(formatear(strings))

0    ojala obliguen a pinera a cerrar punta peuco e...
1    pinera para crear base de apoyo moderada a su ...
2    cnnchile memoria  adimark pinera termina su go...
3    ppk y pinera en privado habrian conversado alg...
4     bachelet entregara el gobierno de chile a pinera
dtype: object


## Funciones auxiliares (conteo y ordenar repetidas)

In [126]:
def most_common(str_arr):
  # get an iterable of (item, iterable) pairs
  SL = sorted((x, i) for i, x in enumerate(str_arr))
  list_pairs = []
  #print('SL:', SL)
  groups = itertools.groupby(SL, key=operator.itemgetter(0))
    
  # auxiliary function to get "quality" for an item
  def _auxfun(g):
    item, iterable = g
    count = 0
    min_index = len(str_arr)
    for _, where in iterable:
      count += 1
      min_index = min(min_index, where)
    list_pairs.append((item, count))
    #print('item %r, count %r, minind %r' % (item, count, min_index))
    return count, -min_index

  return max(groups, key=_auxfun)[0], list_pairs

def aplicar_ordenar_str_arr(func, str_arr):
    arr = list(map(func, str_arr))
    common, pairs = most_common(arr)
    pares_filtrados = list(filter(lambda x: len(x[0]) > 3, pairs))
    common_sorted = sorted(pares_filtrados, key=lambda tup: tup[1], reverse=True)
    return common_sorted

## Separando data por clases

In [5]:
datos_interes = datos[datos.Interes == True]
#print(datos_interes.head())
str_interes = datos_interes.ix[:,0]

datos_no_interes = datos[datos.Interes == False]
#print(datos_no_interes.head())
str_no_interes = datos_no_interes.ix[:,0]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


## Stemming por clases

In [34]:
stemmer_es = lambda x: SpanishStemmer().stem(x)

stem_interes_ordenado = pd.Series(aplicar_ordenar_str_arr(stemmer_es,oracionToStrArr(formatear(str_interes))))
stem_no_interes_ordenado = pd.Series(aplicar_ordenar_str_arr(stemmer_es,oracionToStrArr(formatear(str_no_interes))))

#stem_interes_ordenado

In [35]:
#stem_no_interes_ordenado

## Lematización

In [8]:
def create_lemma_dict(filename):
   with open(filename, 'r') as document:
       lemma_dict = {}
       for line in document:
           if line.strip():  # avoid empty lines
               value, key = line.split(None, 1) # 'None' means 'all whitespace', which is the default
               key = key.rstrip() # rstrip() to get rid of \r and \n
               lemma_dict[key] = value # adding the flections as keys to the dict
               lemma_dict[value] = value # adding also the base word as a key
   return lemma_dict

def query_word(lemma_dict):
   word = input("\nDame una palabra en español -> ")
   try:
      lemma = lemma_dict[word]
      print("__your happy lemma is__: {}".format(lemma))
   except KeyError:
      print("This word is not in the dictionary!")
   return query_word(lemma_dict)

def lemmatiser(dict):
    def lookup(word):
        try:
            lemma = dict[word]
        except:
            lemma = word
        
        return lemma
    
    return lookup

## Lematizando por clases

In [9]:
resource_file = 'lemmatization-es.txt'
lemmatiser_es = lemmatiser(create_lemma_dict(resource_file))

#pdb.set_trace()

datos_interes = datos[datos.Interes == True]
#print(datos_interes.head())
str_interes = datos_interes.ix[:,0]

datos_no_interes = datos[datos.Interes == False]
#print(datos_no_interes.head())
str_no_interes = datos_no_interes.ix[:,0]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':


In [36]:
lema_interes_ordenado = pd.Series(aplicar_ordenar_str_arr(lemmatiser_es, oracionToStrArr(formatear(str_interes))))
lema_no_interes_ordenado = pd.Series(aplicar_ordenar_str_arr(lemmatiser_es, oracionToStrArr(formatear(str_no_interes))))

#lema_interes_ordenado

In [37]:
#lema_no_interes_ordenado

## Extracción de características usando sklearn

In [155]:
stop_words_es = np.genfromtxt('stop_words_es.txt', dtype='str')
stop_words_es = formatear(pd.Series(stop_words_es))
stop_words_es = list(map(lambda x: x, stop_words_es))

In [156]:
tf_vectorizer = CountVectorizer(min_df=2, max_features=20, stop_words=stop_words_es)
data = formatear(strings)
#pdb.set_trace()
dtm_tf = tf_vectorizer.fit_transform(data)

## Aplicando Latent Dirichlet Allocation

In [149]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [157]:
lda = LatentDirichletAllocation(n_components=3, max_iter=10,
                                learning_method='online',
                                learning_offset=10,
                                random_state=1)

In [158]:
n_top_words = 6
lda.fit(dtm_tf)
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topics in LDA model:
Topic #0: pinera bachelet sebastian chile gobierno cierre
Topic #1: pinera peuco punta pais bachelet piera
Topic #2: pinera mando macri cambio domingo sebastian



In [159]:
pyLDAvis.sklearn.prepare(lda, dtm_tf, tf_vectorizer)

/home/martin/.local/lib/python3.5/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      47.213885        1       1  0.114465  0.089178
1      26.461519        1       2 -0.157783  0.031550
2      26.324596        1       3  0.043318 -0.120728, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
12    Default   16.000000       peuco   16.000000  20.0000  20.0000
17    Default   16.000000       punta   16.000000  19.0000  19.0000
9     Default   10.000000       mando   10.000000  18.0000  18.0000
7     Default    8.000000       macri    8.000000  17.0000  17.0000
13    Default    9.000000       piera    9.000000  16.0000  16.0000
2     Default    7.000000      cambio    7.000000  15.0000  15.0000
11    Default   10.000000        pais   10.000000  14.0000  14.0000
5     Default    7.000000     domingo    7.000000  13.0000  13.0000
16    Default   13.000000  presidente   13.000000  12.0000  12.0000
0     Default    6.000000    asuncion    6.000000  11.0000  11.0000
15    Default    6.000000        pone    6.000000  10.0000  10.0000
6     Default   14.000000    gobierno   14.000000   9.0000   9.0000
14    Default  122.000000      pinera  122.000000   8.0000   8.0000
18    Default    5.000000     reunion    5.000000   7.0000   7.0000
3     Default   20.000000       chile   20.000000   6.0000   6.0000
19    Default   23.000000   sebastian   23.000000   5.0000   5.0000
10    Default    4.000000       nuevo    4.000000   4.0000   4.0000
1     Default   31.000000    bachelet   31.000000   3.0000   3.0000
4     Default    4.000000      cierre    4.000000   2.0000   2.0000
8     Default    5.000000      manana    5.000000   1.0000   1.0000
6      Topic1   13.368937    gobierno   14.704337   0.6553  -2.5001
4      Topic1    3.368805      cierre    4.676998   0.4224  -3.8785
14     Topic1   87.498271      pinera  122.132293   0.4170  -0.6214
3      Topic1   14.416880       chile   20.541457   0.3964  -2.4247
19     Topic1   14.668261   sebastian   23.997845   0.2582  -2.4074
8      Topic1    3.101205      manana    5.152413   0.2428  -3.9613
1      Topic1   18.315384    bachelet   31.502036   0.2082  -2.1853
10     Topic1    2.629122       nuevo    4.710197   0.1674  -4.1264
15     Topic1    0.726434        pone    6.115934  -1.3800  -5.4127
18     Topic1    0.416264     reunion    5.632539  -1.8545  -5.9695
...       ...         ...         ...         ...      ...      ...
4      Topic2    0.468824      cierre    4.676998  -0.9707  -5.2716
10     Topic2    0.359097       nuevo    4.710197  -1.2444  -5.5382
0      Topic2    0.433719    asuncion    6.412155  -1.3641  -5.3494
7      Topic2    0.558342       macri    8.312778  -1.3711  -5.0968
15     Topic2    0.334484        pone    6.115934  -1.5766  -5.6092
3      Topic2    1.024408       chile   20.541457  -1.6689  -4.4900
5      Topic2    0.345764     domingo    7.510997  -1.7489  -5.5761
2      Topic2    0.340095      cambio    7.609443  -1.7784  -5.5926
9      Topic2    0.372148       mando   10.596481  -2.0195  -5.5025
6      Topic2    0.348389    gobierno   14.704337  -2.4131  -5.5685
9      Topic3    9.848387       mando   10.596481   1.2615  -2.2216
2      Topic3    6.914970      cambio    7.609443   1.2390  -2.5752
5      Topic3    6.741423     domingo    7.510997   1.2266  -2.6006
7      Topic3    7.403332       macri    8.312778   1.2188  -2.5069
0      Topic3    5.579076    asuncion    6.412155   1.1955  -2.7899
15     Topic3    5.055015        pone    6.115934   1.1441  -2.8885
18     Topic3    2.084479     reunion    5.632539   0.3406  -3.7744
10     Topic3    1.721978       nuevo    4.710197   0.3284  -3.9654
16     Topic3    4.524727  presidente   13.012636   0.2783  -2.9993
19     Topic3    6.616929   sebastian   23.997845   0.0463  -2.6192
3      Topic3    5.100168       chile   20.541457  -0.0585  -2.8796
11     Topic3  